In [2]:
import lancedb
db_path = "D:\\projects\\LLM-Explore\\src\\agent\\contact-query\\tmp\\contact_vectors.lancedb"
# --- 1. 连接/创建 LanceDB 数据库 ---
db = lancedb.connect(db_path)
print(f"LanceDB connected at: {db.uri}")

LanceDB connected at: D:\projects\LLM-Explore\src\agent\contact-query\tmp\contact_vectors.lancedb


In [4]:
# --- 2. 打开表 ---
table_name = "contact_table" # 你之前创建的表名
try:
    table = db.open_table(table_name)
    print(f"Successfully opened table: '{table.name}'")
    print(f"Table schema: {table.schema}")
    print(f"Number of rows in table: {len(table)}")
except Exception as e:
    print(f"Error opening table '{table_name}': {e}")
    print(f"Available tables: {db.table_names()}")
    exit()

Successfully opened table: 'contact_table'
Table schema: vector: fixed_size_list<item: float>[1024]
  child 0, item: float
id: string
payload: string
Number of rows in table: 5


In [5]:
import json

def read_and_decode_table(db, table_name):
    table = db.open_table(table_name)
    df = table.to_pandas()
    
    if 'payload' not in df.columns:
        raise ValueError("DataFrame中缺少'payload'列")
    
    # 解析payload字符串为字典
    df['parsed_payload'] = df['payload'].apply(
        lambda x: json.loads(x) if isinstance(x, str) else x
    )
    
    # 直接提取已解码的字段值
    df['name_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    
    df['content_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('content') if isinstance(x, dict) else None
    )
    
    # 可选：删除临时解析列
    df = df.drop(columns=['parsed_payload'])
    
    return df

In [6]:
data = read_and_decode_table(db, table_name)

In [7]:
data

,vector,id,payload,name_cn,content_cn
0,"[0.009817782, -0.044066403, 0.03745331, -0.010...",ef1e42d58ce63bd4933db8c4d3ae60f4,"{""name"": null, ""meta_data"": {}, ""content"": ""<p...",None,<p>项目名称：包头钢铁（集团）有限责任公司新体系2×500m2烧结余热利用项目</p><p...
1,"[0.011855226, -0.06790031, 0.073237166, 0.0390...",0b51f93442b44ab0c6cf519f9cd22de9,"{""name"": null, ""meta_data"": {}, ""content"": ""<p...",None,"<p><a id=""_Toc246043249""></a>买方合同号：BTYR-E-2012..."
2,"[-0.013582038, -0.043055642, 0.08844583, 0.030...",c9de1fe98741dd805b18ffdcd0b16f8e,"{""name"": null, ""meta_data"": {}, ""content"": ""<p...",None,<p><strong>买方合同编号：BTYR-E-2012-002</strong></p>...
3,"[0.036251247, -0.08604692, 0.046516817, 0.0147...",33ed20b58382926a7c3b0de737979222,"{""name"": null, ""meta_data"": {}, ""content"": ""<p...",None,<p><strong>买方合同编号：BGDSYR-2020-02</strong></p><...
4,"[0.033867214, -0.044725403, 0.02459251, 0.0293...",1fdb558522e1c01775a187dc87b1b4f1,"{""name"": null, ""meta_data"": {}, ""content"": ""<p...",None,<p><strong>买方合同编号：BGYRHS-2023-05</strong></p><...
